## Problem Definition

In this project I will  be developing a text classification model with TensorFlow on movie reviews as positive or negative using the text of the reviews.This is a binary classification problem where the outcome is expected to be positive or negative.

## Data
I’ll walk you through the basic application of transfer learning with TensorFlow Hub and Keras. I will be using the IMDB dataset which contains the text of 50,000 movie reviews from the internet movie database. These are divided into 25,000 assessments for training and 25,000 assessments for testing. The training and test sets are balanced in a way that they contain an equal number of positive and negative reviews

## Importing the necessary libraries
we will start this project of ours by getting the necessary libraries

In [1]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Version:  2.9.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


Although the dataset I am using here is available online to download, but I will simply load the data using TensorFlow. It means you don’t need to download the dataset from any external sources. Now, I will simply load the data and split it into training and test sets:

In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSHXW85/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSHXW85/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSHXW85/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Data Exploration
Let’s have a look at the data to figure out what we are going to work with. I will simply print the first 10 samples from the dataset:

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Now, let’s print the first 10 labels from the data set:

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Building Text Classification Model
To build a model for the task of Text Classification with TensorFlow, I will use a pre-trained model provided by TensorFlow which is known by the name TensorFlow Hub. Let’s first create a Keras layer that uses a TensorFlow Hub model to the embed sentences, and try it out on some sample input:

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067]],
      dtype=float32)>

Now build the model on the complete dataset:



In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Compile The Model
Now, I will compile the model by using the loss function and the adam optimizer:

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train The Text Classification Model
we are going to train the model for 20 epochs in mini-sets of 512 samples. These are 20 iterations on all the samples of the tensors x_train and y_train. During training, we can monitor model loss and accuracy on the 10,000 samples in the validation set:

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 57ms/step - loss: 0.9832 - accuracy: 0.5279 - val_loss: 0.7681 - val_accuracy: 0.5514
Epoch 2/20
30/30 [==============================] - 1s 42ms/step - loss: 0.7350 - accuracy: 0.5603 - val_loss: 0.6969 - val_accuracy: 0.5814
Epoch 3/20
30/30 [==============================] - 1s 44ms/step - loss: 0.6777 - accuracy: 0.6065 - val_loss: 0.6495 - val_accuracy: 0.6311
Epoch 4/20
30/30 [==============================] - 1s 42ms/step - loss: 0.6280 - accuracy: 0.6467 - val_loss: 0.6052 - val_accuracy: 0.6598
Epoch 5/20
30/30 [==============================] - 1s 42ms/step - loss: 0.5877 - accuracy: 0.6716 - val_loss: 0.5725 - val_accuracy: 0.6816
Epoch 6/20
30/30 [==============================] - 1s 43ms/step - loss: 0.5549 - accuracy: 0.6976 - val_loss: 0.5452 - val_accuracy: 0.7061
Epoch 7/20
30/30 [==============================] - 1s 43ms/step - loss: 0.5250 - accuracy: 0.7239 - val_loss: 0.5196 - val_accuracy: 0.7326
Epoch 8/20
30

## Evaluating The Model
And let’s see how the text classification model works. Two values ​​will be returned. Loss and accuracy rate:

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.0841 - accuracy: 0.9738 - 1s/epoch - 24ms/step
loss: 0.084
accuracy: 0.974


So our text classification model got a 97 percent accuracy which is quite good.